# Cooperatives Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql stat_cooperative_created  <<
SELECT date_part('year', founding_date) AS year, COUNT(*)
FROM businesses 
WHERE date(founding_date at time zone 'utc' at time zone 'pst')  > date(current_date - interval '9 years')
GROUP BY date_part('year', founding_date)
ORDER BY date_part('year', founding_date) DESC;

In [ ]:
last_month = datetime.now() - relativedelta(months=1)
filename = os.path.join(os.getcwd(), r'data/')+'cooperative_monthly_stats_for_' + format(last_month, '%B_%Y') +'.csv'

with open(filename, 'w') as f:
    f.write('Number of Cooperatives Created in Last 10 Years:\n')
    stat_cooperative_created.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_cooperative_disslouted  <<
SELECT date_part('year', dissolution_date) AS year
      , COUNT(*) companies_dissoluted
FROM businesses 
WHERE dissolution_date is not null AND date(dissolution_date at time zone 'utc' at time zone 'pst')  > date(current_date - interval '9 years')
GROUP BY date_part('year', dissolution_date)
ORDER BY date_part('year', dissolution_date) DESC;

In [ ]:
df_cooperative_disslouted = stat_cooperative_disslouted.DataFrame()

In [ ]:
with open(filename, 'a') as f:  
    if df_cooperative_disslouted.empty:
        f.write('\n\n\n Number of Cooperatives Dissoluted in Last 10 Years:\n count\n 0\n')
    else: 
        f.write('\n\n\n Number of Cooperatives Dissoluted in Last 10 Years:\n')
        df_cooperative_disslouted.to_csv(f, sep=',', encoding='utf-8', index=False)      
        


In [ ]:
%%sql stat_director_number  <<
SELECT concat('Having One Director:', ' ', count(*))  AS count
FROM (SELECT count(*) AS cnt FROM parties pt, party_roles pr WHERE pt.id=pr.party_id AND pr.cessation_date is null AND pr.role='director' GROUP BY pr.business_id) u
WHERE cnt=1
UNION ALL
SELECT concat('Having Two or Three Directors:', ' ', count(*))  AS count
FROM (SELECT count(*) AS cnt FROM parties pt, party_roles pr WHERE pt.id=pr.party_id AND pr.cessation_date is null AND pr.role='director' GROUP BY pr.business_id) u
WHERE cnt=2 or cnt=3
UNION ALL
SELECT concat('Having Four or More Directors:', ' ', count(*)) AS count
FROM (SELECT count(*) AS cnt FROM parties pt, party_roles pr WHERE pt.id=pr.party_id AND pr.cessation_date is null AND pr.role='director' GROUP BY pr.business_id) u
WHERE cnt>3;

In [ ]:
with open(filename, 'a') as f:  
    f.write('\n\n\n Number of Cooperatives with Directors:\n')
    stat_director_number.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False) 

In [ ]:
%%sql stat_none_bc_cooperative  <<
SELECT count(*) FROM businesses b, parties pt, party_roles pr, addresses a
WHERE b.id = pr.business_id AND pt.id=pr.party_id AND pr.role='director' AND (pt.delivery_address_id=a.id OR pt.mailing_address_id=a.id)
AND lower(a.region) not like 'bc';

In [ ]:
df_none_bc_cooperative = stat_none_bc_cooperative.DataFrame()

In [ ]:
with open(filename, 'a') as f:
    if df_none_bc_cooperative.empty:
        f.write('\n\n\n Number of None BC Businesses:\n count\n 0\n')
    else: 
        f.write('\n\n\n Number of None BC Businesses:\n')
        df_none_bc_cooperative.to_csv(f, sep=',', encoding='utf-8', index=False)   
